In [3]:
# For Colab users
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Install spacytextblob
!pip install spacytextblob
 # Download additional data for TextBlob
!Python-m textblob.download_corpora


/bin/bash: line 1: Python-m: command not found


In [5]:
import pandas as pd
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [6]:
# Load language model
nlp = spacy.load('en_core_web_sm')

In [7]:
# Add extension
nlp.add_pipe('spacytextblob')

In [8]:
dataframe = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Rugby_social_media_comments.txt')
dataframe.head(10)

,Comments
0,Amazing game! Congrats to the champions!
1,That was an intense match. Well-deserved win!
2,"Feeling gutted for the losing team, but the wi..."
3,The champions really showed why they're the be...
4,"Heartbreaking loss, but it was a fair game."
5,No way! That match was totally rigged.
6,The refs were definitely biased. What a joke!
7,What a game! The champions truly showcased the...
8,I’m so proud of our team despite the loss. The...
9,Congratulations to the defending champions! Th...


In [9]:
# Checking for missing values
dataframe.isnull().sum()

Comments    0
dtype: int64

In [10]:
# Text sample to check if function runs as expected
sample = "Hello how are you this fine Sunday morning?"

In [11]:
def clean_text(text):
  text = str(text).lower().strip()

  # Process the text with spaCy
  doc = nlp(text)

  cleaned_tokens = [token.text for token in doc if not token.is_stop and token.is_alpha]
  cleaned_text = ' '.join(cleaned_tokens) # To make the list of strings in cleaned_tokens one complete sentence again

  return cleaned_text

In [12]:
converted_sample = clean_text(sample)
converted_sample

'hello fine sunday morning'

In [13]:
# Create new column with preprocessed comments
dataframe['Cleaned_Comments'] = dataframe['Comments'].apply(clean_text)
dataframe.head()

,Comments,Cleaned_Comments
0,Amazing game! Congrats to the champions!,amazing game congrats champions
1,That was an intense match. Well-deserved win!,intense match deserved win
2,"Feeling gutted for the losing team, but the wi...",feeling gutted losing team winners played bril...
3,The champions really showed why they're the be...,champions showed best impressive performance
4,"Heartbreaking loss, but it was a fair game.",heartbreaking loss fair game


In [14]:
# Grab one comment to check how polarity works
example = dataframe['Cleaned_Comments'][9]
example

'congratulations defending champions performance short spectacular player contributed victory teamwork exemplary tough team lose admit winners deserved thrilling season year'

In [15]:
doc = nlp(example)
polarity = doc._.blob.polarity
polarity

0.11527777777777777

In [16]:
# Function that uses polarity to deduce sentiment behind comment
def analyse_sentiment(text):
  doc = nlp(text)

  polarity = doc._.blob.polarity

  if polarity > 0:
    sentiment = 'Positive'
  elif polarity < 0:
    sentiment = 'Negative'
  else:
    sentiment = 'Neutral'

  return sentiment

In [17]:
# Grabbing 2 comments to use for sentiment analysis
comment1 = dataframe['Cleaned_Comments'][17]
comment2 = dataframe['Cleaned_Comments'][35]

comment1, comment2

('refs need investigated robbery', 'proud team despite loss')

In [18]:
analyse_sentiment(comment1)

'Neutral'

Do not agree that the sentiment is neutral. This can be highlighted as one of the limitations in the basic text processing capabilities of the TextBlob library when used directly with spaCy in the pdf file you submit. For nuanced or sarcastic comments, more sophisticated models should be used.

In [19]:
analyse_sentiment(comment2)

'Positive'

Agreed. This can be highlighted a strength. For clearly positive words like "proud" our model captures the sentiment accurately.

In [20]:
# Extra step: comparing similarity

def compare_similarity(comment1, comment2):
  doc1 = nlp(comment1)
  doc2 = nlp(comment2)
  similarity = doc1.similarity(doc2)

  return similarity

In [21]:
compare_similarity(comment1, comment2)

<ipython-input-20-0c07f88a88c2>:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = doc1.similarity(doc2)


0.4184832119492727